In [1]:
import pandas as pd

In [4]:
df = pd.read_csv('../datasets/dataset_clean.csv')
df.head()

,URUT,R101,R101_NAMA,R105,R108,R111,R201,R201A,R201B,R1301A,...,R1608C3,R1608C4,R1608C5,R1608C6,R1609C1,R1609C2,R1609C3,R1609C4,R1609C5,R1609C6
0,1,11,Aceh,2,1,1,2,0,2,1,...,2,2,2,2,2,2,2,2,2,2
1,2,11,Aceh,2,2,1,4,3,1,1,...,2,2,2,2,2,2,2,2,2,2
2,3,11,Aceh,2,3,1,1,0,1,1,...,2,2,2,2,2,2,2,2,2,2
3,4,11,Aceh,2,4,1,4,3,1,1,...,2,2,2,2,2,2,2,2,2,2
4,5,11,Aceh,2,5,1,7,5,2,1,...,2,2,2,2,2,2,2,2,2,2


In [8]:
# drop kolom yang tidak diperlukan
df.drop(columns=['URUT', 'R101', 'R101_NAMA', 'R105', 'R108', 'R111', 'R201', 'R201A', 'R201B', 'B4K1', 'B4K3', 'B4K4', 'B4K5', 'B4K6', 'B4K7', 'LAMA_TINGG'], inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np


# Asumsi: Kolom target Anda bernama 'target_variable'
# Ganti 'target_variable' dengan nama kolom target yang sebenarnya dari dataset Anda
target_variable = 'target_variable'

# --- Bagian 1: Analisis Korelasi ---
print(f"Menganalisis korelasi dengan '{target_variable}'...")

if target_variable not in df.columns:
    print(f"Error: Kolom target '{target_variable}' tidak ditemukan di dataset Anda.")
    print("Harap periksa nama kolom target dan pastikan sudah benar.")
else:
    # Menghitung korelasi semua kolom numerik dengan target_variable
    # Hanya kolom numerik yang dapat dihitung korelasinya
    numeric_df = df.select_dtypes(include=np.number)
    if target_variable not in numeric_df.columns:
        print(f"Warning: Kolom target '{target_variable}' bukan numerik. Korelasi tidak dapat dihitung.")
    else:
        correlations = numeric_df.corr()[target_variable].sort_values(ascending=False)

        # Menampilkan 50 kolom dengan korelasi terbesar (termasuk korelasi negatif)
        # Ambil 50 teratas berdasarkan nilai absolut korelasi
        top_50_correlations = correlations.reindex(correlations.abs().sort_values(ascending=False).index).head(50)

        print("\n50 Kolom dengan Korelasi Terbesar (berdasarkan nilai absolut) dengan", target_variable, ":")
        print(top_50_correlations)

        # --- Bagian 2: Random Forest Regressor ---
        print("\nMembangun model Random Forest Regressor...")

        # Menyiapkan data untuk model
        # Drop kolom target dari fitur
        features = df.drop(columns=[target_variable])
        labels = df[target_variable]

        # Menangani nilai yang hilang (strategi sederhana: mengisi dengan median)
        # Anda mungkin perlu strategi yang lebih canggih tergantung data Anda
        for col in features.select_dtypes(include=np.number).columns:
            features[col] = features[col].fillna(features[col].median())

        # Untuk fitur non-numerik (kategorikal), bisa dilakukan one-hot encoding
        # Ini adalah contoh sederhana; sesuaikan dengan kebutuhan Anda
        features = pd.get_dummies(features)

        # Pastikan kolom yang dihasilkan dari get_dummies konsisten
        # Jika ada kolom yang hanya muncul di train atau test set, ini bisa menyebabkan masalah
        # Untuk kesederhanaan, kita akan mengasumsikan data Anda cukup bersih

        # Membagi data menjadi training dan testing set
        X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

        # Inisialisasi model Random Forest Regressor
        # Anda bisa menyesuaikan parameter seperti n_estimators, max_depth, dll.
        rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

        # Melatih model
        print("Melatih model...")
        rf_regressor.fit(X_train, y_train)
        print("Model selesai dilatih.")

        # Melakukan prediksi pada test set
        y_pred = rf_regressor.predict(X_test)

        # Mengevaluasi model
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)

        print(f"\nMean Squared Error (MSE): {mse:.2f}")
        print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")

        # Menampilkan pentingnya fitur (feature importance)
        # Ini menunjukkan kolom mana yang paling berkontribusi pada prediksi model
        if hasattr(rf_regressor, 'feature_importances_'):
            importances = pd.Series(rf_regressor.feature_importances_, index=X_train.columns)
            sorted_importances = importances.sort_values(ascending=False)

            print("\n10 Fitur Terpenting (dari Random Forest Regressor):")
            print(sorted_importances.head(10))